In [18]:
import numpy as np
import pandas as pd

### Load ATLAS Data

#### Acceptance

In [19]:
tauDict = {'deltaM30': {0.3 : (9,56), 1.0 : (26,42), 3.0 : (43,28), 10.0 : (60,14), 30.0 : (77,0)},
           'm100' : {0.3 : (9,100), 1.0 : (32,80), 3.0 : (55,60), 10.0 : (78,40), 30.0 : (101,20)}
         }
           

dfs = []
files = {'deltaM30' : './HEPData-ins2080541-v1-csv/Acceptance,R-hadron,compressed.csv',
         'm100' : './HEPData-ins2080541-v1-csv/Acceptance,R-hadron.csv'}
for tag in tauDict:
    for tau,(skip_header,skip_footer) in tauDict[tag].items():
        deltaM30 = np.genfromtxt(files[tag],skip_header=skip_header,delimiter=',',names=True,skip_footer=skip_footer)
        newDF = pd.DataFrame.from_dict({'mLLP' : deltaM30['Mass_GeV']})
        if tag == 'deltaM30':
            newDF['mLSP'] = newDF['mLLP']-30.0
        else:
            newDF['mLSP'] = 100.0
        newDF['Acceptance'] = deltaM30['Acceptance']
        newDF['tau_ns'] = tau
        dfs.append(newDF)

atlasAccDF = pd.concat(dfs, ignore_index=True)

#### Event Level Efficiency

In [20]:
dfs = []
files = {'deltaM30' : './HEPData-ins2080541-v1-csv/Event-levelefficiency,R-hadron,compressed.csv',
         'm100' : './HEPData-ins2080541-v1-csv/Event-levelefficiency,R-hadron.csv'}
for tag in tauDict:
    for tau,(skip_header,skip_footer) in tauDict[tag].items():
        deltaM30 = np.genfromtxt(files[tag],skip_header=skip_header,delimiter=',',names=True,skip_footer=skip_footer)
        newDF = pd.DataFrame.from_dict({'mLLP' : deltaM30['Mass_GeV']})
        if tag == 'deltaM30':
            newDF['mLSP'] = newDF['mLLP']-30.0
        else:
            newDF['mLSP'] = 100.0
        newDF['Event Efficiency'] = deltaM30['Eventlevel_Efficiency']
        newDF['tau_ns'] = tau
        dfs.append(newDF)

atlasEvtEffDF = pd.concat(dfs, ignore_index=True)

#### SR Efficiency

In [21]:
atlasSRs = []
for sr in ['Inclusive_Low','Inclusive_High']:
    sr_tag = sr.split('_')[1]
    dfs = []
    files = {'deltaM30' : './HEPData-ins2080541-v1-csv/Efficiency,SR-%s,R-hadron,compressed.csv'%sr,
            'm100' : './HEPData-ins2080541-v1-csv/Efficiency,SR-%s,R-hadron.csv'%sr}
    for tag in tauDict:
        for tau,(skip_header,skip_footer) in tauDict[tag].items():
            deltaM30 = np.genfromtxt(files[tag],skip_header=skip_header,delimiter=',',names=True,skip_footer=skip_footer)
            newDF = pd.DataFrame.from_dict({'mLLP' : deltaM30['Mass_GeV']})
            if tag == 'deltaM30':
                newDF['mLSP'] = newDF['mLLP']-30.0
            else:
                newDF['mLSP'] = 100.0
            newDF['(SR-%s - no mass Window)' %sr_tag] = deltaM30['Efficiency_%s' %sr]
            newDF['tau_ns'] = tau
            dfs.append(newDF)

    atlasSRs.append(pd.concat(dfs, ignore_index=True))
atlasSREffDF = pd.merge(*atlasSRs)

In [22]:
atlasDF = pd.merge(atlasAccDF, atlasEvtEffDF,how='inner')
atlasDF = pd.merge(atlasDF, atlasSREffDF,how='inner')

In [23]:
orderColumns = ['mLLP','mLSP','tau_ns']
atlasDF.sort_values(orderColumns,inplace=True,
                ascending=[True,True,True],ignore_index=True)
res = atlasDF.reindex(columns=orderColumns+[c for c in atlasDF.columns if c not in orderColumns])

In [24]:
atlasDF.to_pickle('gluinoTables.pcl')